In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from brian2 import *
from brian2tools import *
import numpy as np
import pandas as pd


# Customize matplotlib
matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'font.size': 22,
        'ytick.color' : 'black',
        "xtick.color" : 'black',
        'axes.labelcolor' : 'black',
        'axes.edgecolor' : 'black'
	}
)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [3]:
n_points = 200
n_samples = 1
vec_tauA = []
for tauA_value in np.linspace(0.1, 100, n_points):
	for n in range(n_samples):
		vec_tauA.append(tauA_value)

vec_tauA = np.array(vec_tauA)

EL = -60*mV
EE = 0*mV
EI = -80*mV
alpha = 1
tauN = 100*ms
tauI = 10*ms
tauL = 20*ms
rate = 5

eqs = '''
dv/dt = ((EL - v) + gA*(EE - v) + gI*(EI - v))/tauL : volt
dgA/dt = -gA/tauA : 1
dgI/dt = -gI/tauI : 1
tauA : second
'''

wE_list = [0.1, 1.0, 10]
wI = 0.8
mean_V_ta = {'tauA':vec_tauA, 0.1:np.zeros(n_points*n_samples), 1:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples)}
var_V_ta = {'tauA':vec_tauA, 0.1:np.zeros(n_points*n_samples), 1:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples)}

for wE in wE_list:
	print(wE, end='\t')
	for i, tauA_value in enumerate(vec_tauA):
		start_scope()
		defaultclock.dt = 0.01*ms
		print(i/size(vec_tauA), end=' ')
		G = NeuronGroup(1, eqs, method='rk4')
		GPE = PoissonGroup(400, rates=rate*Hz)
		GPI = PoissonGroup(100, rates=rate*Hz)

		SE = Synapses(GPE, G, 'w : 1', on_pre='gA += w')
		SI = Synapses(GPI, G, 'w : 1', on_pre='gI += w')
		SE.connect()
		SI.connect()

		G.v = EL
		SE.w = wE
		SI.w = wI

		G.tauA = tauA_value*ms

		run(5000*ms)

		M = StateMonitor(G, 'v', record=True)
		
		run(20000*ms)

		mean_V_ta[wE][i] = np.mean(M.v[0]/mV)
		var_V_ta[wE][i] = np.var(M.v[0]/mV)


	print()

10	0.0 0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 0.235 0.24 0.245 0.25 0.255 0.26 0.265 0.27 0.275 0.28 0.285 0.29 0.295 0.3 0.305 0.31 0.315 0.32 0.325 0.33 0.335 0.34 0.345 0.35 0.355 0.36 0.365 0.37 0.375 0.38 0.385 0.39 0.395 0.4 0.405 0.41 0.415 0.42 0.425 0.43 0.435 0.44 0.445 0.45 0.455 0.46 0.465 0.47 0.475 0.48 0.485 0.49 0.495 0.5 0.505 0.51 0.515 0.52 0.525 0.53 0.535 0.54 0.545 0.55 0.555 0.56 0.565 0.57 0.575 0.58 0.585 0.59 0.595 0.6 0.605 0.61 0.615 0.62 0.625 0.63 0.635 0.64 0.645 0.65 0.655 0.66 0.665 0.67 0.675 0.68 0.685 0.69 0.695 0.7 0.705 0.71 0.715 0.72 0.725 0.73 0.735 0.74 0.745 0.75 0.755 0.76 0.765 0.77 0.775 0.78 0.785 0.79 0.795 0.8 0.805 0.81 0.815 0.82 0.825 0.83 0.835 0.84 0.845 0.85 0.855 0.86 0.865 0.87 0.875 0.88 0.885 0.89 0.895 0.9 0.905 0.91 0

In [4]:
data_directory_path = "../../Data/V_data/"

# Convert to pandas DataFrame
df_mean_V_ta = pd.DataFrame.from_dict(mean_V_ta)
df_var_V_ta = pd.DataFrame.from_dict(var_V_ta)


# Save Dataframe do csv file
df_mean_V_ta.to_csv(f"{data_directory_path}mean_V_ta_wE_10.dat")
df_var_V_ta.to_csv(f"{data_directory_path}var_V_ta_wE_10.dat")

In [21]:
mean_V_ta['tauA'].size

2500